In [2]:
from flask import Flask, request, render_template, jsonify
import random
from apscheduler.schedulers.background import BackgroundScheduler
import Non_ITSM_ticket_raising
from predict import *
import pandas as pd
import create_db
import mail_send
import Non_ITSM_ticket_updation

app = Flask(__name__)

@app.route('/newt/<subject>/<description>/<requester_name>/<macid>', methods = ['GET','POST'])
def newt(subject, description, requester_name, macid):
    print(subject, description, requester_name, macid)
    r_json = Non_ITSM_ticket_raising.raiseTicket("sunny singh","hardik.seth@prithvi.ai","9911750445",description)
    print(r_json)
    t_id = r_json['Id']
#     t_id = "456"
    print(t_id)
    predict(macid)
    mail_send.raiseticket('hardik.seth@prithvi.ai','issue',str(t_id))
    return str(t_id)

@app.route('/inoutserver/<macid>')
def inoutserver(macid):
    df = create_db.getuser_Data()
    df = df.loc[df['MAC_ID']==macid]
    print(df)
    ins = df['IN_SERVER'].values[0]
    outs = df['OUT_SERVER'].values[0]
    print(ins,outs)
    data = {
        'inserver':str(ins),
        'outserver':str(outs)
    }
    return jsonify({'inserver':str(ins),'outserver':str(outs)})

@app.route('/userdetails/<key>/<Hostname>/<IP_Address>/<MAC_ID>/<Serial_Number>/<OS_Version>/<Laptop_Desktop>/<IN_SERVER>/<OUT_SERVER>/<Direct_Printers>/<User_Name>', methods=['GET', 'POST'])
def userdetail(key,Hostname, IP_Address, MAC_ID, Serial_Number, OS_Version, Laptop_Desktop, IN_SERVER, OUT_SERVER, Direct_Printers, User_Name):
    if key == 'old':
        print(Hostname, IP_Address, MAC_ID, Serial_Number, OS_Version, Laptop_Desktop, IN_SERVER, OUT_SERVER, Direct_Printers, User_Name)
        df = create_db.getuser_Data()
        if len(df.loc[df["MAC_ID"] == MAC_ID])>0:
            create_db.adduser_update(Hostname, IP_Address, MAC_ID, Serial_Number, OS_Version, Laptop_Desktop, IN_SERVER, OUT_SERVER, Direct_Printers, User_Name)
        else:
            create_db.adduser_details(Hostname, IP_Address, MAC_ID, Serial_Number, OS_Version, Laptop_Desktop, IN_SERVER, OUT_SERVER, Direct_Printers, User_Name)
        return "done"
    elif key == 'new':
        query = "insert into userdetails values('1','1','"+str(MAC_ID)+"','1','1','1','outlook.office365.com','smtp.office365.com','1','1');"
        res = create_db.createnewuser(query)
        return 'done'

@app.route('/oldt/<tid>', methods = ['GET', 'POST'])
def oldt(tid):
    query = "select * from tickets where `Incident ID` = '"+str(tid)+"';"
    df = create_db.fetchquery(query)
    text = 'You had '+str(df.loc[df["Incident ID"] == str(tid)].Description.values[0])+' issue and status is '+str(df.loc[df["Incident ID"] == str(tid)].Status.values[0])
    return text

@app.route('/upt/<tid>', methods = ['GET','POST'])
def upt(tid):
    outp = create_db.updatenew(tid)
#     df = create_db.get_data()
    query = "select * from tickets where `Incident ID` ='"+str(tid)+"';"
    print(query)
    df = create_db.fetchquery(query)
    print(df["Issue_Class"][0])
    Non_ITSM_ticket_updation.updatetickets(str(tid),'Resolved','Issue has been successfully resolved')
    mail_send.updateticket("hardik.seth@prithvi.ai",df["Issue_Class"][0],str(tid))
    if outp == 'Updated':
        return 'Ticket resolved successfully'
    else:
        return 'Some Error While Resolving Issue'
    return 'Ticket resolved successfully'


@app.route('/know/<macid>',methods = ['GET','POST'])
def know(macid):
    query = "select * from tickets where MAC_ID = '"+macid+"';"
    df = create_db.fetchquery(query)
    df = df.to_json()
    return df

@app.route('/getalluniqueid', methods = ['GET','POST'])
def getall():
    query = 'select MAC_ID from userdetails'
    df = create_db.fetchquery(query)
    df = df.to_json()
    return df
if __name__ == '__main__':
    app.run(host = '0.0.0.0', port = 7006)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:7006/ (Press CTRL+C to quit)
139.5.254.242 - - [28/Jul/2020 08:21:05] "GET /getalluniqueid HTTP/1.1" 200 -
139.5.254.242 - - [28/Jul/2020 08:21:05] "GET /inoutserver/aae3c695ce7f11ea968c08979870b990 HTTP/1.1" 200 -


          Hostname   IP_Address                            MAC_ID  \
0  LAPTOP-KE0MC8QD  192.168.0.5  aae3c695ce7f11ea968c08979870b990   

  Serial_Number                 OS_Version Laptop_Desktop  \
0         12345  Windows-10-10.0.18362-SP0           desk   

               IN_SERVER          OUT_SERVER Direct_Printers User_Name  
0  outlook.office365.com  smtp.office365.com               2     hseth  
outlook.office365.com smtp.office365.com


139.5.254.242 - - [28/Jul/2020 08:21:06] "GET /userdetails/old/LAPTOP-KE0MC8QD/192.168.0.5/aae3c695ce7f11ea968c08979870b990/12345/Windows-10-10.0.18362-SP0/desk/outlook.office365.com/smtp.office365.com/2/hseth HTTP/1.1" 200 -


LAPTOP-KE0MC8QD 192.168.0.5 aae3c695ce7f11ea968c08979870b990 12345 Windows-10-10.0.18362-SP0 desk outlook.office365.com smtp.office365.com 2 hseth
update userdetails set Hostname='LAPTOP-KE0MC8QD', IP_Address='192.168.0.5' , Serial_Number='12345' , OS_Version='Windows-10-10.0.18362-SP0' , Laptop_Desktop='desk', Direct_Printers='2' , User_Name='hseth' where MAC_ID = 'aae3c695ce7f11ea968c08979870b990';


139.5.254.242 - - [28/Jul/2020 08:21:35] "GET /know/aae3c695ce7f11ea968c08979870b990 HTTP/1.1" 200 -
139.5.254.242 - - [28/Jul/2020 08:21:43] "GET /oldt/12414 HTTP/1.1" 200 -
139.5.254.242 - - [28/Jul/2020 08:21:55] "GET /oldt/12415 HTTP/1.1" 200 -


In [1]:
import Non_ITSM_ticket_raising
r_json = Non_ITSM_ticket_raising.raiseTicket("sunny singh","hardik.seth@prithvi.ai","9911750445","printer_not working properly")
print(r_json['Id'])

12408


In [2]:
from predict import *
predict("12")

[{'complain': {'Id': 12408, 'ComplainNo': 'AFS10012407', 'ContactPerson': 'sunny singh', 'ContactPersonEmail': 'hardik.seth@prithvi.ai', 'ContactPersonPhoneNumber': '9911750445', 'ProblemDescription': 'printer_not working properly', 'SerialNo': '123', 'ComplainStatusId': 1, 'complainstatus': {'Id': 1, 'ComplainStatus': 'New'}, 'ComplainRegDate': '2020-07-25T19:09:52.873', 'location': {'Id': 1, 'LocationName': 'NAIKKANAL TRICHUR (Manappuram Finance Ltd)', 'Office': 'THRISSUR-07', 'District': 'THRISSUR - 2 ', 'Location': 'KERALA', 'Address': 'Thrissur Naikanal Door No. XXIX / 267/6, Brothers Complex, Naikkanal, Thrissur Dt., K.L Pin - 680 001, Phone - 0487-2335544  / 8590394363', 'VendorId': 1, 'VendorList': None}, 'LocationMasterId': 1, 'ComplainType': {'Id': 1, 'ComplainType': 'Service Request'}, 'ComplainTypeId': 1, 'ModelType': {'Id': 1, 'ModelTypeName': 'Safe Camera'}, 'ModelTypeId': 1, 'ProblemType': {'Id': 1, 'ProblemTypeName': 'Desktop Problem'}, 'ProblemTypeId': 1, 'ProblemSubTy

/home/dev/.local/lib/python3.6/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.20.4 when using version 0.21.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/dev/.local/lib/python3.6/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.20.4 when using version 0.21.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/dev/.local/lib/python3.6/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.20.4 when using version 0.21.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
